In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [63]:
from sklearn import preprocessing, linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import confusion_matrix, cohen_kappa_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [10]:
df = pd.read_csv("D:\Redi_DataCircle2\survey_results_public.csv")

In [11]:
df = df.dropna(subset=['ConvertedCompYearly'])

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48019 entries, 1 to 89171
Data columns (total 84 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ResponseId                           48019 non-null  int64  
 1   Q120                                 48019 non-null  object 
 2   MainBranch                           48019 non-null  object 
 3   Age                                  48019 non-null  object 
 4   Employment                           48007 non-null  object 
 5   RemoteWork                           47940 non-null  object 
 6   CodingActivities                     47918 non-null  object 
 7   EdLevel                              48019 non-null  object 
 8   LearnCode                            47935 non-null  object 
 9   LearnCodeOnline                      38414 non-null  object 
 10  LearnCodeCoursesCert                 20744 non-null  object 
 11  YearsCode                        

In [13]:
df['ConvertedCompYearly'].value_counts()

ConvertedCompYearly
64254.0     784
53545.0     615
150000.0    585
200000.0    558
74963.0     557
           ... 
31494.0       1
859.0         1
77904.0       1
74917.0       1
16917.0       1
Name: count, Length: 8784, dtype: int64

In [8]:
df['RemoteWork'].value_counts()

RemoteWork
Remote                                  21138
Hybrid (some remote, some in-person)    20276
In-person                                6526
Name: count, dtype: int64

In [ ]:
#Check if important for salary
df["RemoteWork"]
df["Age"] - categorize
df["Employment"] - simplify
df["EdLevel"]
df["YearsCode"] or "YearsCodePro" before change rthe columns less then or more then  (should be normalized or categorized for ML)
"YearsCodePro" make more sence cause its exp without studying. check if its always less or not. also  check with df["WorkExp"]  and choose one of them
df["DevType"] question is if there is enought for some special roles
df["OrgSize"] categorise
df["PurchaseInfluence"] categorise
df["Country"] - group firstly to bigger groups (EU, EastEurope, others)
df["ProfessionalTech"] group firstly 
df["Industry"]  

In [5]:
df["EdLevel"].value_counts()

EdLevel
Bachelor’s degree (B.A., B.S., B.Eng., etc.)                                          22540
Master’s degree (M.A., M.S., M.Eng., MBA, etc.)                                       13045
Some college/university study without earning a degree                                 5654
Professional degree (JD, MD, Ph.D, Ed.D, etc.)                                         2297
Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)     2169
Associate degree (A.A., A.S., etc.)                                                    1596
Something else                                                                          496
Primary/elementary school                                                               222
Name: count, dtype: int64

In [6]:
df["EdLevel"].value_counts().keys()

Index(['Bachelor’s degree (B.A., B.S., B.Eng., etc.)',
       'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)',
       'Some college/university study without earning a degree',
       'Professional degree (JD, MD, Ph.D, Ed.D, etc.)',
       'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
       'Associate degree (A.A., A.S., etc.)', 'Something else',
       'Primary/elementary school'],
      dtype='object', name='EdLevel')

In [29]:
df["ProfessionalTech"].value_counts()

ProfessionalTech
None of these                                                                                                                                                                                           3346
DevOps function;Microservices;Automated testing;Observability tools;Developer portal or other central places to find tools/services;Continuous integration (CI) and (more often) continuous delivery    2114
DevOps function;Microservices;Automated testing;Observability tools;Continuous integration (CI) and (more often) continuous delivery                                                                    1881
DevOps function;Microservices;Automated testing;Continuous integration (CI) and (more often) continuous delivery                                                                                        1298
DevOps function;Automated testing;Continuous integration (CI) and (more often) continuous delivery                                                                 

In [35]:
df["Knowledge_1"].value_counts()

Knowledge_1
Agree                         19812
Strongly agree                15502
Neither agree nor disagree     3593
Disagree                       2603
Strongly disagree              1025
Name: count, dtype: int64

In [5]:
def drop_from_df(df, labels, axis=1):
    return df.drop(labels, axis=axis, errors='ignore')

In [6]:
df = drop_from_df(df, ['Q120', 'TechList','BuyNewTool','SurveyLength','SurveyEase'])

In [37]:
replacement_map = {
    "I am a developer by profession": "dev",
    "I am not primarily a developer, but I write code sometimes as part of my work/studies": "code, but not dev",
    "I am learning to code": "learning",
    "I code primarily as a hobby": "hobby",
    "I used to be a developer by profession, but no longer am": "was developer",
    "None of these": "other"
}
df['MainBranch'] = df['MainBranch'].replace(replacement_map)

In [40]:
df['MainBranch'].value_counts()

MainBranch
dev                  44354
code, but not dev     3665
Name: count, dtype: int64

In [25]:
df["CompTotal"].value_counts()#.head(40) - do a visualisation of distribution

CompTotal
100000.0     1355
60000.0      1290
120000.0     1240
150000.0     1150
80000.0      1066
             ... 
165.0           1
182700.0        1
2810000.0       1
49280.0         1
182333.0        1
Name: count, Length: 3797, dtype: int64

Knowledge_1	I have interactions with people outside of my immediate team.

Knowledge_2	Knowledge silos prevent me from getting ideas across the organization (i.e., one individual or team has information that isn't shared with others)	

Knowledge_3	I can find up-to-date information within my organization to help me do my job.

Knowledge_4	I am able to quickly find answers to my questions with existing tools and resources.

Knowledge_5	I know which system or resource to use to find information and answers to questions I have.

Knowledge_6	I often find myself answering questions that I’ve already answered before.	

Knowledge_7	Waiting on answers to questions often causes interruptions and disrupts my workflow.

Knowledge_8	I feel like I have the tools and/or resources to quickly understand and work on any area of my company's code/system/platform.

Positive Questions (where "Strongly agree" indicates a better work experience or environment):
Knowledge_1: Positive. If someone strongly agrees, it means they have good interactions beyond their immediate team, suggesting a positive, collaborative environment.
Knowledge_3: Positive. Being able to find up-to-date information to do one's job is a positive aspect of the work environment.
Knowledge_4: Positive. Quickly finding answers with existing tools and resources indicates efficiency and good support systems.
Knowledge_5: Positive. Knowing which system or resource to use for information indicates clarity and good organizational structure.
Knowledge_8: Positive. Having the necessary tools and resources for understanding and working on company code/system/platform suggests a well-equipped and supportive environment.

Negative Questions (where "Strongly agree" indicates a less favorable work experience or environment):
Knowledge_2: Negative. Agreeing strongly suggests that knowledge silos are a significant issue, hindering the sharing of ideas and information across the organization.
Knowledge_6: Negative. If someone often finds themselves answering the same questions, it can imply inefficiencies or communication gaps.
Knowledge_7: Negative. Waiting on answers causing interruptions is a sign of inefficiency and can lead to frustration and disrupted workflow.

Frequency - not relevant
1. Needing help from people outside of your immediate team?  
2. Interacting with people outside of your immediate team?  

3. Encountering knowledge silos (where one individual or team has information that's not
shared or distributed with other individuals or teams) at work?


df["OrgSize"]

In [14]:
df["OrgSize"].value_counts()

OrgSize
20 to 99 employees                                    10310
100 to 499 employees                                   9518
10,000 or more employees                               6099
1,000 to 4,999 employees                               5702
2 to 9 employees                                       4318
10 to 19 employees                                     3886
500 to 999 employees                                   3450
5,000 to 9,999 employees                               2063
Just me - I am a freelancer, sole proprietor, etc.     2054
I don’t know                                            582
Name: count, dtype: int64

In [15]:
from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=[
    'Just me - I am a freelancer, sole proprietor, etc.', 
    '2 to 9 employees', 
    '10 to 19 employees', 
    '20 to 99 employees', 
    '100 to 499 employees', 
    '500 to 999 employees', 
    '1,000 to 4,999 employees', 
    '5,000 to 9,999 employees', 
    '10,000 or more employees', 
], ordered=True)

df['OrgSize'] = df['OrgSize'].astype(cat_type)
df['OrgSize_encoded'] = df['OrgSize'].cat.codes
df.loc[df['OrgSize'].isna() | (df['OrgSize'] == 'I don’t know'), 'OrgSize_encoded'] = -1
df['OrgSize'] = df['OrgSize'].astype(cat_type)

In [16]:
df['OrgSize_encoded'].value_counts()

OrgSize_encoded
 3    10310
 4     9518
 8     6099
 6     5702
 1     4318
 2     3886
 5     3450
 7     2063
 0     2054
-1      619
Name: count, dtype: int64

df['Age']

In [17]:
df['Age'].value_counts()

Age
25-34 years old       22163
35-44 years old       13848
18-24 years old        4922
45-54 years old        4866
55-64 years old        1795
65 years or older       339
Under 18 years old       56
Prefer not to say        30
Name: count, dtype: int64

In [18]:
# Label-Encoding für Altersgruppen
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['Encoded_Age'] = label_encoder.fit_transform(df['Age'])

In [19]:
label_encoder = LabelEncoder()
df['Encoded_Age'].value_counts()

Encoded_Age
1    22163
2    13848
0     4922
3     4866
4     1795
5      339
7       56
6       30
Name: count, dtype: int64

df["Employment"]

In [26]:
df["Employment"].value_counts().head(20)

Employment
Employed, full-time                                                                             39301
Independent contractor, freelancer, or self-employed                                             3906
Employed, full-time;Independent contractor, freelancer, or self-employed                         3291
Employed, part-time                                                                               906
Independent contractor, freelancer, or self-employed;Employed, part-time                          240
Employed, full-time;Employed, part-time                                                           154
Employed, full-time;Independent contractor, freelancer, or self-employed;Employed, part-time      109
I prefer not to say                                                                                34
Retired                                                                                            33
Independent contractor, freelancer, or self-employed;Retired           

In [27]:
from itertools import chain
split_values = df['Employment'].str.split(';')

flattened_values = list(chain.from_iterable(split_values.dropna()))
value_counts = pd.Series(flattened_values).value_counts()
value_counts

Employed, full-time                                     42866
Independent contractor, freelancer, or self-employed     7568
Employed, part-time                                      1415
Retired                                                    66
I prefer not to say                                        34
Name: count, dtype: int64

In [28]:
df_exploded = df.assign(Employment=df['Employment'].str.split(';')).explode('Employment')
one_hot = pd.get_dummies(df_exploded['Employment'])
one_hot = one_hot.groupby(level=0).sum()

df = df.join(one_hot)
df.head()


,ResponseId,Q120,MainBranch,Age,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,...,SurveyLength,SurveyEase,ConvertedCompYearly,OrgSize_encoded,Encoded_Age,"Employed, full-time","Employed, part-time",I prefer not to say,"Independent contractor, freelancer, or self-employed",Retired
1,2,I agree,I am a developer by profession,25-34 years old,"Employed, full-time",Remote,Hobby;Contribute to open-source projects;Boots...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;Friend or fam...,Formal documentation provided by the owner of ...,...,Appropriate in length,Easy,285000.0,1,1,1,0,0,0,0
2,3,I agree,I am a developer by profession,45-54 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby;Professional development or self-paced l...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;On the job tr...,Formal documentation provided by the owner of ...,...,Appropriate in length,Easy,250000.0,7,3,1,0,0,0,0
3,4,I agree,I am a developer by profession,25-34 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Colleague;Friend or family member;Other online...,Formal documentation provided by the owner of ...,...,Appropriate in length,Easy,156000.0,4,1,1,0,0,0,0
4,5,I agree,I am a developer by profession,25-34 years old,"Employed, full-time;Independent contractor, fr...",Remote,Hobby;Contribute to open-source projects;Profe...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Online Courses or Certi...,Formal documentation provided by the owner of ...,...,Appropriate in length,Neither easy nor difficult,23456.0,3,1,1,0,0,1,0
5,6,I agree,I am a developer by profession,35-44 years old,"Employed, full-time",Remote,Hobby;Professional development or self-paced l...,Some college/university study without earning ...,Books / Physical media;Colleague;Online Course...,Formal documentation provided by the owner of ...,...,Appropriate in length,Neither easy nor difficult,96828.0,4,2,1,0,0,0,0


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48019 entries, 1 to 89171
Data columns (total 91 columns):
 #   Column                                                Non-Null Count  Dtype   
---  ------                                                --------------  -----   
 0   ResponseId                                            48019 non-null  int64   
 1   Q120                                                  48019 non-null  object  
 2   MainBranch                                            48019 non-null  object  
 3   Age                                                   48019 non-null  object  
 4   Employment                                            48007 non-null  object  
 5   RemoteWork                                            47940 non-null  object  
 6   CodingActivities                                      47918 non-null  object  
 7   EdLevel                                               48019 non-null  object  
 8   LearnCode                                          

Knowledge to points

In [111]:
def calculate_knowledge_score(df):
    positive_points = {
        "Strongly agree": 5,
        "Agree": 4,
        "Neither agree nor disagree": 3,
        "Disagree": 2,
        "Strongly disagree": 1,
        pd.NA: 0}
    negative_points = {
        "Strongly agree": 1,
        "Agree": 2,
        "Neither agree nor disagree": 3,
        "Disagree": 4,
        "Strongly disagree": 5,
        pd.NA: 0}

    positive_questions = ["Knowledge_1", "Knowledge_3", "Knowledge_4", "Knowledge_5", "Knowledge_8"]
    negative_questions = ["Knowledge_2", "Knowledge_6", "Knowledge_7"]

    for col in df.columns:
        if col in positive_questions:
            df[col] = df[col].map(positive_points)
        elif col in negative_questions:
            df[col] = df[col].map(negative_points)

    df['Knowledge_Score'] = df[positive_questions + negative_questions].sum(axis=1)

    return df

In [112]:
calculate_knowledge_score(df)

,ResponseId,MainBranch,Age,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,LearnCodeCoursesCert,...,ProfessionalTech,Industry,ConvertedCompYearly,OrgSize_encoded,"Employed, full-time","Employed, part-time",I prefer not to say,"Independent contractor, freelancer, or self-employed",Retired,Knowledge_Score
1,2,I am a developer by profession,25-34 years old,"Employed, full-time",Remote,Hobby;Contribute to open-source projects;Boots...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;Friend or fam...,Formal documentation provided by the owner of ...,Other,...,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",285000.0,1,1,0,0,0,0,29.0
2,3,I am a developer by profession,45-54 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby;Professional development or self-paced l...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;On the job tr...,Formal documentation provided by the owner of ...,NaN,...,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",250000.0,7,1,0,0,0,0,28.0
3,4,I am a developer by profession,25-34 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Colleague;Friend or family member;Other online...,Formal documentation provided by the owner of ...,NaN,...,Automated testing;Continuous integration (CI) ...,NaN,156000.0,4,1,0,0,0,0,33.0
4,5,I am a developer by profession,25-34 years old,"Employed, full-time;Independent contractor, fr...",Remote,Hobby;Contribute to open-source projects;Profe...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Online Courses or Certi...,Formal documentation provided by the owner of ...,Other;Codecademy;edX,...,Microservices;Automated testing;Observability ...,Other,23456.0,3,1,0,0,1,0,23.0
5,6,I am a developer by profession,35-44 years old,"Employed, full-time",Remote,Hobby;Professional development or self-paced l...,Some college/university study without earning ...,Books / Physical media;Colleague;Online Course...,Formal documentation provided by the owner of ...,Other,...,DevOps function;Microservices;Observability to...,Other,96828.0,4,1,0,0,0,0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89162,89163,I am a developer by profession,35-44 years old,"Employed, full-time",Remote,I don’t code outside of work,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",On the job training;Other online resources (e....,Formal documentation provided by the owner of ...,NaN,...,Automated testing;Developer portal or other ce...,"Information Services, IT, Software Development...",50719.0,7,1,0,0,0,0,24.0
89166,89167,I am a developer by profession,18-24 years old,"Employed, full-time;Independent contractor, fr...",In-person,Contribute to open-source projects;Professiona...,Some college/university study without earning ...,Books / Physical media;Hackathons (virtual or ...,Formal documentation provided by the owner of ...,Codecademy;Udemy,...,DevOps function;Microservices;Continuous integ...,Financial Services,16917.0,3,1,0,0,1,0,26.0
89167,89168,I am a developer by profession,18-24 years old,"Employed, full-time",Remote,Hobby;Bootstrapping a business;Professional de...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Online Courses or Certification;Other online r...,Formal documentation provided by the owner of ...,edX;Udemy;Coursera,...,Automated testing,Retail and Consumer Services,15752.0,3,1,0,0,0,0,25.0
89170,89171,I am a developer by profession,25-34 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Bootstrapping a business;Freelance/contract work,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Colleague;Hackathons (virtual or in-person);On...,Formal documentation provided by the owner of ...,Other,...,Automated testing;Observability tools;

In [113]:
df = drop_from_df(df, ['Knowledge_1', 'Knowledge_2','Knowledge_3','Knowledge_4','Knowledge_5','Knowledge_6','Knowledge_7','Knowledge_8'])

In [114]:
display(df['TimeSearching'].value_counts(),
        df['TimeAnswering'].value_counts() )

TimeSearching
30-60 minutes a day           12382
15-30 minutes a day            8902
60-120 minutes a day           5759
Less than 15 minutes a day     2877
Over 120 minutes a day         2268
Name: count, dtype: int64

TimeAnswering
15-30 minutes a day           10397
30-60 minutes a day            9938
Less than 15 minutes a day     6085
60-120 minutes a day           4279
Over 120 minutes a day         1427
Name: count, dtype: int64

Country

In [115]:
unique_countries = df['Country'].unique()
unique_countries

array(['United States of America', 'Philippines',
       'United Kingdom of Great Britain and Northern Ireland', 'Finland',
       'Australia', 'Netherlands', 'Germany', 'Sweden', 'France',
       'Albania', 'Nigeria', 'Spain', 'South Africa', 'Brazil',
       'Portugal', 'Italy', 'Bangladesh', 'Canada', 'Argentina',
       'Switzerland', 'Sri Lanka', 'Lithuania', 'Serbia', 'Cyprus',
       'India', 'Egypt', 'Latvia', 'Russian Federation', 'Greece',
       'Austria', 'Norway', 'Singapore', 'Turkey', 'Croatia', 'Poland',
       'Iran, Islamic Republic of...', 'Kosovo', 'Nepal', 'Slovenia',
       'China', 'Belgium', 'Romania', 'Denmark', 'Hungary',
       'Republic of Korea', 'Viet Nam', 'Israel', 'Ukraine', 'Nomadic',
       'Dominican Republic', 'Isle of Man', 'Estonia', 'Morocco',
       'The former Yugoslav Republic of Macedonia', 'Uzbekistan',
       'Luxembourg', 'Indonesia', 'Ireland', 'Georgia', 'Benin', 'Japan',
       'Venezuela, Bolivarian Republic of...', 'Bahrain', 'Tajikis

In [116]:
grouped_data = df.groupby('Country')['ConvertedCompYearly'].agg(['mean', 'median', 'count'])
grouped_data.sort_values(by='count', ascending=False).head()

,mean,median,count
Country,,,
United States of America,170783.961199,150000.0,11881
Germany,82457.479093,74963.0,3970
United Kingdom of Great Britain and Northern Ireland,95948.670429,81931.0,3544
Canada,142136.677435,89222.0,2105
India,27346.822805,19023.0,1868


In [118]:
countries_with_low_counts = grouped_data[grouped_data['count'] < 250].index #assuming that we haave not enough representatives to talk about meaningfull data.

df['Region'] = None
df.loc[df['Country'].isin(countries_with_low_counts), 'Region'] = 'Other'

In [119]:
countries_to_exclude = df[df['Region'] == 'Other']['Country'].unique()

grouped_data_excluding_other = grouped_data[~grouped_data.index.isin(countries_to_exclude)] # Filter out other countries from grouped_data

countries_by_mean = grouped_data_excluding_other.sort_values(by='median', ascending=False).head(35)
countries_by_mean


,mean,median,count
Country,,,
United States of America,170783.961199,150000.0,11881
Switzerland,132553.690332,122064.5,662
Israel,114575.031915,112257.0,470
Ireland,108026.028070,96381.0,285
Australia,172617.624277,95851.0,1211
Denmark,101836.885659,93173.0,516
Canada,142136.677435,89222.0,2105
United Kingdom of Great Britain and Northern Ireland,95948.670429,81931.0,3544
New Zealand,109678.277778,78913.0,414


In [120]:
countries_by_mean = grouped_data_excluding_other.sort_values(by='count', ascending=False).head(35)
countries_by_mean

,mean,median,count
Country,,,
United States of America,170783.961199,150000.0,11881
Germany,82457.479093,74963.0,3970
United Kingdom of Great Britain and Northern Ireland,95948.670429,81931.0,3544
Canada,142136.677435,89222.0,2105
India,27346.822805,19023.0,1868
France,65101.007773,56757.0,1801
Netherlands,85128.483847,74963.0,1362
Poland,64335.517378,57518.5,1266
Brazil,52883.154952,30431.0,1252


In [ ]:
United States of America	170783.961199	150000.0	11881
Canada	142136.677435	89222.0	2105


Switzerland	132553.690332	122064.5	662
Ireland	108026.028070	96381.0	285
Denmark	101836.885659	93173.0	516
United Kingdom of Great Britain and Northern Ireland	95948.670429	81931.0	3544
Norway	84747.629630	76568.0	486
Netherlands	85128.483847	74963.0	1362
Germany	82457.479093	74963.0	3970
Austria	80699.340866	71000.0	531
Finland	72157.306818	69608.0	440
Belgium	84706.265169	64254.0	445
Sweden	65003.224432	57542.0	1056
France	65101.007773	56757.0	1801

Israel	114575.031915	112257.0	470

Australia	172617.624277	95851.0	1211
New Zealand	109678.277778	78913.0	414

Poland	64335.517378	57518.5	1266
Czech Republic	59613.657201	54592.0	493
Portugal	78275.953975	53545.0	478
Romania	57885.063725	51786.0	408
Spain	59906.971304	49261.0	1150
Ukraine	45102.496437	44420.0	421
Italy	50595.266798	42836.0	1012
Hungary	47006.215686	39808.5	306
Greece	47021.306548	38552.0	336

South Africa	259535.945869	41497.0	351

Mexico	48364.324242	33777.5	330
Brazil	52883.154952	30431.0	1252
Argentina	40352.068592	30000.0	277
Colombia	33809.165468	21506.5	278

Russian Federation	33126.746479	27227.0	426
Turkey	35879.222785	23504.0	395
India	27346.822805	19023.0	1868
Pakistan	15012.566406	5959.0	256

In [135]:
country_region_mapping = {
    'United States of America': 'North American High Income',
    'Canada': 'North American High Income',
    'Israel': 'Middle Eastern High Income',
    'South Africa': 'African Medium Income',
    'India': 'South Asian Lower Income',
    'Pakistan': 'South Asian Lower Income',
    'Australia': 'Oceanian High Income',
    'New Zealand': 'Oceanian High Income',
    'Mexico': 'South American Medium Income',
    'Brazil': 'South American Medium Income',
    'Argentina': 'South American Medium Income',
    'Colombia': 'South American Medium Income',
    'Russian Federation': 'Eurasian Lower-Medium Income',
    'Turkey': 'Eurasian Lower-Medium Income',

    'Switzerland': 'Central & Northern European High Income',
    'Ireland': 'Central & Northern European High Income',
    'Denmark': 'Central & Northern European High Income',
    'United Kingdom': 'Central & Northern European High Income',
    'Norway': 'Central & Northern European High Income',
    'Netherlands': 'Central & Northern European High Income',
    'Germany': 'Central & Northern European High Income',
    'Austria': 'Central & Northern European High Income',
    'Finland': 'Central & Northern European High Income',
    'Belgium': 'Central & Northern European High Income',
    'Sweden': 'Central & Northern European High Income',
    'France': 'Central & Northern European High Income',

    'Poland': 'Central & Southern European Medium Income',
    'Czech Republic': 'Central & Southern European Medium Income',
    'Portugal': 'Central & Southern European Medium Income',
    'Romania': 'Central & Southern European Medium Income',
    'Spain': 'Central & Southern European Medium Income',
    'Ukraine': 'Central & Southern European Medium Income',
    'Italy': 'Central & Southern European Medium Income',
    'Hungary': 'Central & Southern European Medium Income',
    'Greece': 'Central & Southern European Medium Income'
}
df['Region'] = None
df['Region'] = df['Country'].map(country_region_mapping)
df['Region'].fillna('Other', inplace=True)

In [136]:
df['Region'].value_counts()

Region
North American High Income                   13986
Central & Northern European High Income      11554
Other                                         9081
Central & Southern European Medium Income     5870
South American Medium Income                  2137
South Asian Lower Income                      2124
Oceanian High Income                          1625
Eurasian Lower-Medium Income                   821
Middle Eastern High Income                     470
African Medium Income                          351
Name: count, dtype: int64

In [ ]:
df_encoded = pd.get_dummies(df['Region'], prefix='Region')
df = pd.concat([df, df_encoded], axis=1)
df.drop('Region', axis=1, inplace=True)

small_df

In [17]:
df_GER = df[df['Country'] == 'Germany']
df_GER = df_GER[['Age','Employment','RemoteWork','EdLevel','YearsCodePro','DevType','OrgSize','ConvertedCompYearly','Knowledge_Score']]
df_GER = df_GER.rename(columns={'ConvertedCompYearly': 'Salary'})

In [18]:
df_GER = df_GER.dropna(subset=['RemoteWork', 'YearsCodePro', 'DevType', 'OrgSize'])

In [19]:
df_GER.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3942 entries, 15 to 89146
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Age              3942 non-null   object 
 1   Employment       3942 non-null   object 
 2   RemoteWork       3942 non-null   object 
 3   EdLevel          3942 non-null   object 
 4   YearsCodePro     3942 non-null   object 
 5   DevType          3942 non-null   object 
 6   OrgSize          3942 non-null   object 
 7   Salary           3942 non-null   float64
 8   Knowledge_Score  3942 non-null   float64
dtypes: float64(2), object(7)
memory usage: 308.0+ KB


In [24]:
df_USA = df[df['Country'] == 'United States of America'].copy()
df_USA = df_USA[['Age','Employment','RemoteWork','EdLevel','YearsCodePro','DevType','OrgSize','ConvertedCompYearly','Knowledge_Score']]
df_USA = df_USA.rename(columns={'ConvertedCompYearly': 'Salary'})

Model1

In [43]:
df_USA = df[df['Country'] == 'United States of America'].copy()
df_USA = df_USA[['OrgSize_encoded','Encoded_Age', 'WorkExp', "Employed, full-time", "Employed, part-time","I prefer not to say", "Independent contractor, freelancer, or self-employed", "Retired", 'ConvertedCompYearly']] 
df_USA = df_USA.rename(columns={'ConvertedCompYearly': 'Salary'})

In [47]:
df_USA.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8013 entries, 1 to 89156
Data columns (total 9 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   OrgSize_encoded                                       8013 non-null   int8   
 1   Encoded_Age                                           8013 non-null   int32  
 2   WorkExp                                               8013 non-null   float64
 3   Employed, full-time                                   8013 non-null   int64  
 4   Employed, part-time                                   8013 non-null   int64  
 5   I prefer not to say                                   8013 non-null   int64  
 6   Independent contractor, freelancer, or self-employed  8013 non-null   int64  
 7   Retired                                               8013 non-null   int64  
 8   Salary                                                8013 non-

In [45]:
df_USA = df_USA.dropna(subset=["WorkExp"])

In [46]:
df_USA

,OrgSize_encoded,Encoded_Age,WorkExp,"Employed, full-time","Employed, part-time",I prefer not to say,"Independent contractor, freelancer, or self-employed",Retired,Salary
1,1,1,10.0,1,0,0,0,0,285000.0
2,7,3,23.0,1,0,0,0,0,250000.0
3,4,1,7.0,1,0,0,0,0,156000.0
6,6,2,4.0,1,0,0,0,0,135000.0
7,2,1,5.0,1,0,0,0,0,80000.0
...,...,...,...,...,...,...,...,...,...
89141,7,1,11.0,1,0,0,0,0,250000.0
89143,3,1,9.0,1,0,0,0,0,90000.0
89145,2,2,15.0,1,0,0,0,0,45000.0
89154,1,2,8.0,1,0,0,0,0,75000.0


In [50]:
df_USA["dataset"] = np.random.choice(("train", "test"), size=len(df_USA), p=(0.8, 0.2))
features = ['OrgSize_encoded', 'Encoded_Age', 'WorkExp', 'Employed, full-time',
       'Employed, part-time', 'I prefer not to say',
       'Independent contractor, freelancer, or self-employed', 'Retired']

df_train = df_USA.query("dataset=='train'")
df_test = df_USA.query("dataset=='test'")

X_train = df_train[features].to_numpy()
X_test = df_test[features].to_numpy()

y_train = df_train.Salary.to_numpy()
y_true = df_test.Salary.to_numpy()

In [54]:
from sklearn.linear_model import LinearRegression
model1 = LinearRegression().fit(X_train, y_train)


In [55]:
y_predict = model1.predict(X_test)
y_predict

array([125932.87618727, 203415.68201859, 188618.85751348, ...,
       150245.26993161, 141903.45704918, 153145.13383696])

In [59]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_true, y_predict)


63144.02658115305

In [64]:
#Create a function within many Machine Learning Models
def models(X_train,Y_train):
  # Using Logistic Regression Algorithm to the Training Set
  from sklearn.linear_model import LogisticRegression
  log = LogisticRegression(solver='lbfgs', max_iter=100)
  log.fit(X_train, Y_train.ravel())
  
  # Using Linear Regression Algorithm to the Training Set
  linreg = linear_model.LinearRegression()
  linreg.fit(X_train, Y_train)

  #Using KNeighborsClassifier Method of neighbors class to use Nearest Neighbor algorithm
  from sklearn.neighbors import KNeighborsClassifier
  knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
  knn.fit(X_train, Y_train)

  #Using DecisionTreeClassifier of tree class to use Decision Tree Algorithm
  from sklearn.tree import DecisionTreeClassifier
  tree = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
  tree.fit(X_train, Y_train.ravel())

  #Using RandomForestClassifier method of ensemble class to use Random Forest Classification algorithm
  from sklearn.ensemble import RandomForestClassifier
  forest = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
  forest.fit(X_train, Y_train.ravel())
  
  #print model accuracy on the training data.
  print('Logistic Regression Training Accuracy:', (log.score(X_test, y_true))*100)
  print('Linear Regression Training Accuracy:', (linreg.score(X_test, y_true))*100)
  print('K Nearest Neighbor Training Accuracy:', knn.score(X_test, y_true))
  print('Decision Tree Classifier Training Accuracy:', (tree.score(X_test, y_true))*100)
  print('Random Forest Classifier Training Accuracy:', (forest.score(X_test, y_true))*100)

In [66]:
models(X_train,y_train)

c:\Users\Iurii Novachynskyi\anaconda3\envs\CAB\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Training Accuracy: 4.363859864781807
Linear Regression Training Accuracy: 5.50812075292717
K Nearest Neighbor Training Accuracy: 0.020282728948985862
Decision Tree Classifier Training Accuracy: 3.564843269821758
Random Forest Classifier Training Accuracy: 3.0116779348494163


Model2

Model 3

Model 4

In [23]:
df['YearsCodePro'].value_counts()

AttributeError: 'Series' object has no attribute 'key'

In [25]:
df_USA = df_USA.dropna(subset=['RemoteWork', 'YearsCodePro', 'DevType', 'OrgSize'])

In [26]:
df_USA.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11811 entries, 1 to 89156
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Age              11811 non-null  object 
 1   Employment       11811 non-null  object 
 2   RemoteWork       11811 non-null  object 
 3   EdLevel          11811 non-null  object 
 4   YearsCodePro     11811 non-null  object 
 5   DevType          11811 non-null  object 
 6   OrgSize          11811 non-null  object 
 7   Salary           11811 non-null  float64
 8   Knowledge_Score  11811 non-null  float64
dtypes: float64(2), object(7)
memory usage: 922.7+ KB


In [27]:
corr_matrix = df.corr()['ConvertedCompYearly']

ValueError: could not convert string to float: '25-34 years old'

In [46]:
display (df["Frequency_1"].value_counts()
         , df["Frequency_2"].value_counts()
         , df["Frequency_3"].value_counts()
         )

Frequency_1
1-2 times a week     25528
Never                10707
3-5 times a week      4100
6-10 times a week      847
10+ times a week       734
Name: count, dtype: int64

Frequency_2
1-2 times a week     18930
3-5 times a week      9809
10+ times a week      4794
6-10 times a week     4383
Never                 4009
Name: count, dtype: int64

Frequency_3
1-2 times a week     21470
Never                12107
3-5 times a week      5125
6-10 times a week     1280
10+ times a week      1072
Name: count, dtype: int64

In [20]:
''''
-can write a function that look for some words and create a boolean column
LearnCode
LearnCodeOnline
df["LearnCodeCoursesCert"]
df["LanguageHaveWorkedWith"]
DatabaseHaveWorkedWith
PlatformHaveWorkedWith
WebframeHaveWorkedWith
MiscTechHaveWorkedWith
ToolsTechHaveWorkedWith
NEWCollabToolsHaveWorkedWith
 SurveyLength                         86485 non-null  object 
 82  SurveyEase                           86554 non-null  object 
 83  ConvertedCompYearly''''

NameError: name 'LearnCode' is not defined

In [ ]:
#check CompTotal  if  is the same as if df["ConvertedCompYearly"] and df["Currency"]. look for outliers, mistakes, not correct convertations and other anomalies